In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import time
import os
import re
from itertools import chain
from transformers import BertTokenizer
PRETRAINED_MODEL_NAME = "bert-base-uncased" #英文pretrain(不區分大小寫)
print(torch.__version__)
#1.3.1

1.4.0


# 載入BERT token:

In [2]:
# get pre-train tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
vocab = tokenizer.vocab
print("dict size", len(vocab))

# see some token and index mapping
import random
random_tokens = random.sample(list(vocab), 10)
random_ids = [vocab[t] for t in random_tokens]

print("{0:20}{1:15}".format("token", "index"))
print("-" * 25)
for t, id in zip(random_tokens, random_ids): #隨便看幾個字
    print("{0:15}{1:10}".format(t, id))

dict size 30522
token               index          
-------------------------
basin                6403
leads                5260
[unused937]           942
commonwealth         5663
intimacy            20893
oldest               4587
##xley              20959
slant               27474
killers             15978
several              2195


In [3]:
from torch.utils.data import Dataset,random_split

TAG_RE = re.compile(r'<[^>]+>')
def preprocess html tags
    sentence = TAG_RE.sub('', sen)
    # Remove punctuations and numbers
    sentence = re.sub('_text(sen):
    # Removing[^a-zA-Z]', ' ', sentence)
    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence


In [4]:
os.getcwd()

'/home/zihjie/Test/IMDB_bert_test'

In [4]:
def readIMDB(path, seg):
    classes = ['pos', 'neg']
    data = []
    for label in classes:
        files = os.listdir(os.path.join(path, seg, label))
        for file in files:
            with open(os.path.join(path, seg, label, file), 'r', encoding='utf-8') as rf:
                review = rf.read().replace('\n', '')
                if label == 'pos':
                    data.append([preprocess_text(review), 1])
                elif label == 'neg':
                    data.append([preprocess_text(review), 0])
    return data

In [5]:
#修改
def readIMDB(path, seg):
    classes = ['pos', 'neg']
    data = []
    for label in classes:
        files = os.listdir(os.path.join(path, seg, label))
        for file in files:
            try:
                rf =  open(os.path.join(path, seg, label, file), 'r', encoding='utf-8') 
                review = rf.read().replace('\n', '')
                if label == 'pos':
                    data.append([preprocess_text(review), 1])
                elif label == 'neg':
                    data.append([preprocess_text(review), 0])
            except:
                print('maybe some error')
    return data

In [6]:
label_map = {0: 'neg', 1: 'pos'}

#create Dataset
class MyDataset(Dataset):
    def __init__(self, mode, tokenizer):
        assert mode in ["train", "test"]  
        self.mode = mode
        self.df = readIMDB('/home/zihjie/Test/IMDB_bert_test/aclImdb',mode) #its list [['text1',label],['text2',label],...]
        self.len = len(self.df)
        self.maxlen = 300 #限制文章長度(若你記憶體夠多也可以不限)
        self.tokenizer = tokenizer  # we will use BERT tokenizer
    
    # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self, idx):
        origin_text = self.df[idx][0]
        if self.mode == "test":
            text_a = self.df[idx][0]
            text_b = None  #for natural language inference
            #label_tensor = None #in our case, we have label
            label_id = self.df[idx][1]
            label_tensor = torch.tensor(label_id)
        else:     
            text_a = self.df[idx][0]
            text_b = None  #for natural language inference
            label_id = self.df[idx][1]
            label_tensor = torch.tensor(label_id)
            
        # 建立第一個句子的 BERT tokens
        word_pieces = ["[CLS]"]
        tokens_a = self.tokenizer.tokenize(text_a)
        word_pieces += tokens_a[:self.maxlen] + ["[SEP]"]
        len_a = len(word_pieces)
        
        if text_b is not None:
            tokens_b = self.tokenizer.tokenize(text_b)
            word_pieces += tokens_b + ["[SEP]"]
            len_b = len(word_pieces) - len_a
               
        # 將整個 token 序列轉換成索引序列
        ids = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(ids)
        
        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        if text_b is None:
            segments_tensor = torch.tensor([1] * len_a,dtype=torch.long)
        elif text_b is not None:
            segments_tensor = torch.tensor([0] * len_a + [1] * len_b,dtype=torch.long)
        
        return (tokens_tensor, segments_tensor, label_tensor, origin_text)
    
    def __len__(self):
        return self.len

In [7]:
# initialize Dataset
trainset = MyDataset("train", tokenizer=tokenizer)
testset = MyDataset("test", tokenizer=tokenizer)


#split val from trainset
val_size = int(trainset.__len__()*0.04) #比對LSTM 切出1000筆當validation
trainset, valset = random_split(trainset,[trainset.__len__()-val_size,val_size])
print('trainset size:' ,trainset.__len__())
print('valset size:',valset.__len__())
print('testset size: ',testset.__len__())

maybe some error
maybe some error
maybe some error
maybe some error
trainset size: 24000
valset size: 1000
testset size:  25000


In [11]:
trainset[5]

(tensor([  101,  1045,  2061,  2293,  2023,  3185,  1996,  7284,  2003,  2307,
          2005, 13433,  2243, 12256,  3185,  1996,  1039,  5856,  3504,  2061,
         12476,  2293,  1996,  2189,  1999,  1996,  3185,  2061,  2307,  2027,
          2921,  1996,  2887,  2189,  2004,  2005,  1996,  2466,  2049,  2307,
          2009,  2038,  2307,  3110,  1997,  6860,  8292,  2571,  5638,  2003,
          2200, 10140,  1998,  3928, 13433,  2243, 12256,  6683,  2003,  2428,
          2307,  1999,  2023,  3185,  1998,  2066,  2010,  6860,  2007,  3520,
          1996,  2069,  2518,  2134,  2066,  2001, 24086, 10841,  2638,  3311,
          2002,  2074,  3402, 16949,  2039,  7126,  6683,  2522,  2978,  1998,
          3727,  2027,  2071,  2031,  2081,  2010,  2112,  1999,  1996,  3185,
          2210,  7046,  2021,  3452, 12476,  3185,  2064,  3524,  2000,  2219,
          1996,  3915,  2544,  2006,  4966,   102]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [10]:
# 隨便選一個樣本
sample_idx = 10

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor, segments_tensor, label_tensor,origin_text = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())

print('token:\n',tokens,'\n')
print('origin_text:\n',origin_text,'\n')
print('label:',label_map[int(label_tensor.numpy())],'\n')
print('tokens_tensor:\n',tokens_tensor,'\n')
print('segment tensor:\n',segments_tensor)

token:
 ['[CLS]', 'well', 'when', 'before', 'saw', 'this', 'film', 'really', 'wasn', 'sure', 'whether', 'it', 'would', 'be', 'my', 'cup', 'of', 'tea', 'how', 'wrong', 'was', 'thought', 'that', 'this', 'was', 'one', 'of', 'the', 'best', 'films', 've', 'watched', 'for', 'very', 'long', 'time', 'real', 'family', 'classic', 'the', 'story', 'of', 'young', 'eva', '##cu', '##ee', 'and', 'his', 'new', 'foster', 'dad', 'this', 'film', 'tick', '##s', 'all', 'the', 'boxes', 've', 'not', 'read', 'the', 'book', 'maybe', 'that', 'a', 'good', 'thing', 'meant', 'enjoyed', 'the', 'film', 'more', 'but', 'with', 'regards', 'to', 'the', 'story', 'really', 'can', 'think', 'of', 'any', 'bad', 'points', 'hence', 'scoring', 'it', 'out', 'of', 'and', 'hardly', 'ever', 'think', 'anything', 'warrant', '##s', 'top', 'marks', 'by', 'the', 'time', 'william', 'proclaimed', 'can', 'ride', 'my', 'bike', 'dad', 'was', 'sobbing', 'my', 'heart', 'out', 'anyone', 'who', 'seen', 'it', 'will', 'understand', 'm', 'sure', 're

In [14]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

""""
create_mini_batch(samples)吃上面定義的mydataset
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""
#collate_fn: 如何將多個樣本的資料連成一個batch丟進 model
#截長補短後要限制attention只注意非pad 的部分
def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 訓練集有 labels
    if samples[0][2] is not None:
        label_ids = torch.stack([s[2] for s in samples])
    else:
        label_ids = None
    
    # zero pad到該batch下最長的長度
    tokens_tensors = pad_sequence(tokens_tensors, batch_first=True)
    segments_tensors = pad_sequence(segments_tensors,batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape,dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids



# 初始化一個每次回傳 batch size 個訓練樣本的 DataLoader
# 利用 'collate_fn' 將 list of samples 合併成一個 mini-batch
BATCH_SIZE = 16
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE,collate_fn=create_mini_batch,shuffle=True)
valloader = DataLoader(valset, batch_size=BATCH_SIZE,collate_fn=create_mini_batch,shuffle=False)
testloader = DataLoader(testset, batch_size=BATCH_SIZE,collate_fn=create_mini_batch,shuffle=False)

data = next(iter(trainloader))
tokens_tensors, segments_tensors, masks_tensors, label_ids = data
print(tokens_tensors)
print(segments_tensors)
print(masks_tensors)
print(label_ids)

tensor([[  101,  2023,  3185,  ...,     0,     0,     0],
        [  101,  1996,  3585,  ...,     0,     0,     0],
        [  101, 17081,  2003,  ...,     0,     0,     0],
        ...,
        [  101,  2026,  2434,  ...,  2000,  4109,   102],
        [  101,  2023,  3185,  ...,     0,     0,     0],
        [  101,  2023,  2003,  ...,     0,     0,     0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
tensor([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0])


In [15]:
from transformers import BertForSequenceClassification

NUM_LABELS = 2

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=NUM_LABELS)


print("""
name      module
--------------------""")

for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print("{:10}{}".format(name,n) )
    else:
        print("{:10} {}".format(name, module))


name      module
--------------------
bert      embeddings
bert      encoder
bert      pooler
dropout    Dropout(p=0.1, inplace=False)
classifier Linear(in_features=768, out_features=2, bias=True)


In [38]:
torch.cuda.is_available()

True

In [11]:

%%time
from sklearn.metrics import accuracy_score

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:",device)
model = model.to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
EPOCHS = 10

for epoch in range(EPOCHS):
    correct = 0
    #total = 0
    train_loss , val_loss = 0.0 , 0.0
    train_acc, val_acc = 0, 0
    n, m = 0, 0
    model.train()
    for data in trainloader:
        n += 1
        tokens_tensors, segments_tensors,masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)
        # outputs 的順序是 "(loss), logits, (hidden_states), (attentions)"
        
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        
        #get prediction and calulate acc
        logits = outputs[1]
        _, pred = torch.max(logits.data, 1)
        train_acc += accuracy_score(pred.cpu().tolist() , labels.cpu().tolist())

        # 紀錄當前 batch loss
        train_loss += loss.item()
    
    #validation
    with torch.no_grad():
        model.eval()
        for data in valloader:
            m += 1
            tokens_tensors, segments_tensors,masks_tensors, labels = [t.to(device) for t in data]
            val_outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)
            
            logits = val_outputs[1]
            _, pred = torch.max(logits.data, 1)
            val_acc += accuracy_score(pred.cpu().tolist() , labels.cpu().tolist())
            val_loss += val_outputs[0].item()

    print('[epoch %d] loss: %.4f, acc: %.4f, val loss: %4f, val acc: %4f' %
          (epoch+1, train_loss/n, train_acc/n, val_loss/m,  val_acc/m  ))

print('Done')

device: cuda:0
[epoch 1] loss: 0.2744, acc: 0.8830, val loss: 0.231500, val acc: 0.919643
[epoch 2] loss: 0.1521, acc: 0.9447, val loss: 0.221421, val acc: 0.914683
[epoch 3] loss: 0.0874, acc: 0.9704, val loss: 0.278775, val acc: 0.918651
[epoch 4] loss: 0.0501, acc: 0.9843, val loss: 0.294110, val acc: 0.918651
[epoch 5] loss: 0.0360, acc: 0.9890, val loss: 0.353868, val acc: 0.915675
[epoch 6] loss: 0.0287, acc: 0.9916, val loss: 0.437387, val acc: 0.898810
[epoch 7] loss: 0.0214, acc: 0.9936, val loss: 0.432030, val acc: 0.900794
[epoch 8] loss: 0.0200, acc: 0.9938, val loss: 0.416535, val acc: 0.906746
[epoch 9] loss: 0.0167, acc: 0.9949, val loss: 0.433508, val acc: 0.909722
[epoch 10] loss: 0.0116, acc: 0.9966, val loss: 0.508724, val acc: 0.912698
Done
CPU times: user 1h 14min 38s, sys: 23min 46s, total: 1h 38min 24s
Wall time: 1h 38min 25s


In [ ]:
from sklearn.metrics import confusion_matrix

true=[]
predictions=[]
with torch.no_grad():
    model.eval()
    for data in testloader:
        tokens_tensors, segments_tensors,masks_tensors, labels = [t.to(device) for t in data]
        val_outputs = model(input_ids=tokens_tensors, 
                    token_type_ids=segments_tensors, 
                    attention_mask=masks_tensors, 
                    labels=labels)

        logits = val_outputs[1]
        _, pred = torch.max(logits.data, 1)
        true.extend(labels.cpu().tolist())
        predictions.extend(pred.cpu().tolist())


c = confusion_matrix(true, predictions)
print(c)
accuracy_score(predictions,true)    